<a href="https://colab.research.google.com/github/ArturHugo/PLN-2022-1/blob/main/Atividade03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Atividade 03 - Recuperação de textos
Grupo:

- Artur Hugo cunha Pereira, 180030400
- Gabriel da Silva Corvino Nogueira, 180113330 


In [72]:
import re
import pandas as pd
import nltk
import numpy as np

In [73]:
# NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Corpus Reuters

In [74]:
nltk.download('reuters')
from nltk.corpus import reuters

cats = reuters.categories()
print("Reuters has %d categories:\n%s" % (len(cats), cats))

fileids = reuters.fileids()

categories = []
text = []

for file in fileids:
    categories.append(reuters.categories(file))
    text.append(reuters.raw(file))

df = pd.DataFrame({'ids':fileids, 'categories':categories, 'text':text})
df

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


Reuters has 90 categories:
['acq', 'alum', 'barley', 'bop', 'carcass', 'castor-oil', 'cocoa', 'coconut', 'coconut-oil', 'coffee', 'copper', 'copra-cake', 'corn', 'cotton', 'cotton-oil', 'cpi', 'cpu', 'crude', 'dfl', 'dlr', 'dmk', 'earn', 'fuel', 'gas', 'gnp', 'gold', 'grain', 'groundnut', 'groundnut-oil', 'heat', 'hog', 'housing', 'income', 'instal-debt', 'interest', 'ipi', 'iron-steel', 'jet', 'jobs', 'l-cattle', 'lead', 'lei', 'lin-oil', 'livestock', 'lumber', 'meal-feed', 'money-fx', 'money-supply', 'naphtha', 'nat-gas', 'nickel', 'nkr', 'nzdlr', 'oat', 'oilseed', 'orange', 'palladium', 'palm-oil', 'palmkernel', 'pet-chem', 'platinum', 'potato', 'propane', 'rand', 'rape-oil', 'rapeseed', 'reserves', 'retail', 'rice', 'rubber', 'rye', 'ship', 'silver', 'sorghum', 'soy-meal', 'soy-oil', 'soybean', 'strategic-metal', 'sugar', 'sun-meal', 'sun-oil', 'sunseed', 'tea', 'tin', 'trade', 'veg-oil', 'wheat', 'wpi', 'yen', 'zinc']


,ids,categories,text
0,test/14826,[trade],ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RI...
1,test/14828,[grain],CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STO...
2,test/14829,"[crude, nat-gas]",JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWA...
3,test/14832,"[corn, grain, rice, rubber, sugar, tin, trade]",THAI TRADE DEFICIT WIDENS IN FIRST QUARTER\n ...
4,test/14833,"[palm-oil, veg-oil]",INDONESIA SEES CPO PRICE RISING SHARPLY\n Ind...
...,...,...,...
10783,training/999,"[interest, money-fx]",U.K. MONEY MARKET SHORTAGE FORECAST REVISED DO...
10784,training/9992,[earn],KNIGHT-RIDDER INC &lt;KRN> SETS QUARTERLY\n Q...
10785,training/9993,[earn],TECHNITROL INC &lt;TNL> SETS QUARTERLY\n Qtly...
10786,training/9994,[earn],NATIONWIDE CELLULAR SERVICE INC &lt;NCEL> 4TH ...


# Preprocessamento dos dados

In [75]:
# Helper functions
def tokenize(string):
  # Remove special characters
  res = re.sub(r'&lt;|\\n|>', ' ', string.lower())
  # Runs through nltk tokenizer to finish tokenization
  res = nltk.tokenize.word_tokenize(res)
  return res

def filter_stopwords(word_array, stop_list):
  # If word is in stop_list it is filtered out
  res = [word for word in word_array if word not in stop_list]
  return res

In [76]:
texts_df = df['text']
texts_df = texts_df.sample(frac=0.1)
texts_df

1723    SOUTHLAND CORP RAISED CRUDE OIL POSTINGS 50 CT...
6023    &lt;FAR WEST INDUSTRIES INC> YEAR NET\n  Shr 2...
1727    EVERGOOD PRODUCTS CORP &lt;EVGD.O> 1986 YEAR L...
4904    FCS LABORATORIES &lt;FCSI> ENDS MERGER TALKS\n...
5137    FLORIDA COMMERCIAL BANKS INC &lt;FLBK> QTLY DI...
                              ...                        
6085    BUSH &lt;BSH> REVISES 4TH QTR, YEAR RESULTS UP...
9326    PARISIAN INC &lt;PASN> 4TH QTR JAN 31 NET\n  S...
1041    PLANTRONICS INC &lt;PLX> 3RD QTR MARCH 28 NET\...
8280    AMERICAN CYANAMID &lt;ACY> BUYS U.K. COMPANY\n...
2790    ASPEN RIBBONS INC &lt;ARIB.O> 1ST QTR NET\n  S...
Name: text, Length: 1618, dtype: object

In [77]:
texts_df = texts_df.apply(tokenize)
texts_df

1723    [southland, corp, raised, crude, oil, postings...
6023    [far, west, industries, inc, year, net, shr, 2...
1727    [evergood, products, corp, evgd.o, 1986, year,...
4904    [fcs, laboratories, fcsi, ends, merger, talks,...
5137    [florida, commercial, banks, inc, flbk, qtly, ...
                              ...                        
6085    [bush, bsh, revises, 4th, qtr, ,, year, result...
9326    [parisian, inc, pasn, 4th, qtr, jan, 31, net, ...
1041    [plantronics, inc, plx, 3rd, qtr, march, 28, n...
8280    [american, cyanamid, acy, buys, u.k., company,...
2790    [aspen, ribbons, inc, arib.o, 1st, qtr, net, s...
Name: text, Length: 1618, dtype: object

In [78]:
# Remove stop words
stop_list = nltk.corpus.stopwords.words('english')
stop_list.extend(['.', '\'\'', '``', '(', ')', ',', '\'s']) # Additional stopwords
texts_df = texts_df.apply(lambda text: filter_stopwords(text, stop_list))
texts_df

1723    [southland, corp, raised, crude, oil, postings...
6023    [far, west, industries, inc, year, net, shr, 2...
1727    [evergood, products, corp, evgd.o, 1986, year,...
4904    [fcs, laboratories, fcsi, ends, merger, talks,...
5137    [florida, commercial, banks, inc, flbk, qtly, ...
                              ...                        
6085    [bush, bsh, revises, 4th, qtr, year, results, ...
9326    [parisian, inc, pasn, 4th, qtr, jan, 31, net, ...
1041    [plantronics, inc, plx, 3rd, qtr, march, 28, n...
8280    [american, cyanamid, acy, buys, u.k., company,...
2790    [aspen, ribbons, inc, arib.o, 1st, qtr, net, s...
Name: text, Length: 1618, dtype: object

In [79]:
# Stemming
stemmer = nltk.stem.porter.PorterStemmer()
texts_df = texts_df.apply(lambda text: [stemmer.stem(word) for word in text])
texts_df

1723    [southland, corp, rais, crude, oil, post, 50, ...
6023    [far, west, industri, inc, year, net, shr, 23,...
1727    [evergood, product, corp, evgd.o, 1986, year, ...
4904    [fc, laboratori, fcsi, end, merger, talk, fc, ...
5137    [florida, commerci, bank, inc, flbk, qtli, div...
                              ...                        
6085    [bush, bsh, revis, 4th, qtr, year, result, bus...
9326    [parisian, inc, pasn, 4th, qtr, jan, 31, net, ...
1041    [plantron, inc, plx, 3rd, qtr, march, 28, net,...
8280    [american, cyanamid, aci, buy, u.k., compani, ...
2790    [aspen, ribbon, inc, arib.o, 1st, qtr, net, se...
Name: text, Length: 1618, dtype: object

# Representações: BoW e TF-IDF

In [80]:
# Corpus and vocabulary

corpus = []
for text in texts_df:
  for word in text:
    corpus.append(word)

vocab = set(sorted(corpus))

In [81]:
# Counting Bag of Words

bow_count = {}
for i, text in enumerate(texts_df.values):
  bow_count[df['ids'][i]] = dict()
  for word in text:
    if word in bow_count[df['ids'][i]]:
      bow_count[df['ids'][i]][word] += 1
    else:
      bow_count[df['ids'][i]][word] = 1

bow_count_df = pd.DataFrame().from_records(bow_count).fillna(0).T.astype(int)
bow_count_df

,southland,corp,rais,crude,oil,post,50,cts/bbl,wti,19,...,aci,fothergil,honeycomb,wrexham,ribbon,arib.o,"234,504","157,862","4,096,000","3,007,383"
test/14826,2,2,2,2,2,2,2,2,2,2,...,0,0,0,0,0,0,0,0,0,0
test/14828,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
test/14829,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
test/14832,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
test/14833,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test/18357,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
test/18360,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
test/18362,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
test/18363,0,0,0,0,0,0,1,0,0,0,...,1,2,1,1,0,0,0,0,0,0


In [82]:
# TF-IDF

document_freq = {}
for word in vocab:
  document_freq[word] = 0
  for text, word_counts in bow_count.items():
    if word in word_counts:
      document_freq[word] += 1

tf_idf  = {}
n_texts = len(texts_df.values)
for text_id, word_counts in bow_count.items():
  n_words = sum(word_counts.values())
  tf_idf[text_id] = {}
  for word, count in word_counts.items():
    tf  = count/n_words
    idf = np.log(n_texts/document_freq[word])
    tf_idf[text_id][word] = tf*idf

tf_idf_df = pd.DataFrame().from_records(tf_idf).fillna(0).T
tf_idf_df

,southland,corp,rais,crude,oil,post,50,cts/bbl,wti,19,...,aci,fothergil,honeycomb,wrexham,ribbon,arib.o,"234,504","157,862","4,096,000","3,007,383"
test/14826,0.608709,0.129813,0.244406,0.295074,0.208098,0.327705,0.254907,0.545696,0.494821,0.338671,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
test/14828,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
test/14829,0.000000,0.046063,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.120174,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
test/14832,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
test/14833,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test/18357,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
test/18360,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
test/18362,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
test/18363,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.056080,0.000000,0.000000,0.000000,...,0.147779,0.295558,0.147779,0.147779,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Recuperação de informação

In [83]:
# Cosine similarity
def cosine_similarity(v1, v2):
  return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [84]:
# Calcula similaride de uma notícia com todas as demais notícias
def similarity_ranking(text_id, texts_df, sim_function):
  '''
    text_id:      row's id of the text representation in the dataframe.
    texts_df:     dataframe with the texts to rank against.
    sim_function: similarity function to use in the comparisons.

    Calculate the similarity between the text in row with name text_id and 
    the remaining texts in the dataframe texts_df using sim_function.

    Returns a list o text_ids in order of most to least similar.
  '''

  # Calculate similarities
  query_text = texts_df.loc[text_id].values
  similarities_df = texts_df.apply(
      lambda text: sim_function(text.values, query_text),
      axis=1)
  
  # Remove the query text's similarity with itself
  similarities_df = similarities_df.drop(text_id)

  # Sort from most to least similar
  similarities_df = similarities_df.sort_values(ascending=False)

  # Return the similarity ranking with the id of the texts
  return similarities_df.index.to_list()

In [85]:
# Ranking de similaridade usando BoW
bow_count_ranking = bow_count_df.apply(
    lambda text: similarity_ranking(text.name, bow_count_df, cosine_similarity),
    axis=1).to_dict()
print(bow_count_ranking)

# Ranking de similaridades usando TF-IDF
tf_idf_ranking = tf_idf_df.apply(
    lambda text: similarity_ranking(text.name, tf_idf_df, cosine_similarity),
    axis=1).to_dict()
print(tf_idf_ranking)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [86]:
# Limitando o rankings ao top 10
bow_count_top = bow_count_ranking
for text_id in bow_count_top:
  bow_count_top[text_id] = bow_count_top[text_id][0:10]
print(bow_count_top)

tf_idf_top = tf_idf_ranking
for text_id in tf_idf_top:
  tf_idf_top[text_id] = tf_idf_top[text_id][0:10]
print(tf_idf_top)

{'test/14826': ['test/17957', 'test/15762', 'test/17671', 'test/16652', 'test/15300', 'test/15580', 'test/16148', 'test/15061', 'test/15153', 'test/16723'], 'test/14828': ['test/15593', 'test/17837', 'test/15572', 'test/15777', 'test/17703', 'test/14962', 'test/18249', 'test/15551', 'test/15109', 'test/15785'], 'test/14829': ['test/15120', 'test/15230', 'test/18300', 'test/16802', 'test/16055', 'test/15417', 'test/15210', 'test/16808', 'test/16426', 'test/16871'], 'test/14832': ['test/16179', 'test/17757', 'test/15478', 'test/16749', 'test/16123', 'test/18242', 'test/17700', 'test/15976', 'test/16642', 'test/17579'], 'test/14833': ['test/15090', 'test/17617', 'test/16339', 'test/17727', 'test/15194', 'test/17625', 'test/15471', 'test/16264', 'test/17783', 'test/16161'], 'test/14839': ['test/16680', 'test/18086', 'test/15198', 'test/17986', 'test/15528', 'test/18210', 'test/15970', 'test/16472', 'test/16234', 'test/15011'], 'test/14840': ['test/17916', 'test/17805', 'test/16202', 'test/

In [87]:
# Confere se duas listas tem elementos em comum
def common(list_a, list_b):
  set_a = set(list_a)
  set_b = set(list_b)
  if set_a & set_b:
    return True
  else:
    return False

In [88]:
# Verifica a acurácia do ranking de um dicionário de similaridade
def top_accuracies(ranking_dict, original_df):
  '''
    ranking_dict: dictionary of similarity rankings for each text in dataframe.
    original_df:  original dataframe to check categories of the rankings.

    Calculate the how many elements of the top ranking had a category in common
    with their respective query text.

    Returns a dictionary with text ids for keys and their respective accuracy
    for values.
  '''

  accuracies_dict = {}
  for text_id in bow_count_top:
    accuracies_dict[text_id] = 0
    text_categories = original_df[original_df['ids'] == text_id]['categories'].values[0]
    top_ids = bow_count_top[text_id]
    for top_id in top_ids:
      top_categories = original_df[original_df['ids'] == top_id]['categories'].values[0]
      if common(text_categories, top_categories):
        accuracies_dict[text_id] += 1
    accuracies_dict[text_id] /= len(bow_count_top[text_id])

  return accuracies_dict

In [89]:
# Verificar acurácia dos top 10
bow_count_top_accuracies = top_accuracies(bow_count_top, df)
print(bow_count_top_accuracies)

tf_idf_top_accuracies = top_accuracies(tf_idf_top, df)
print(tf_idf_top_accuracies)

{'test/14826': 0.0, 'test/14828': 0.1, 'test/14829': 0.1, 'test/14832': 0.1, 'test/14833': 0.0, 'test/14839': 0.0, 'test/14840': 0.0, 'test/14841': 0.1, 'test/14842': 0.0, 'test/14843': 0.1, 'test/14844': 0.0, 'test/14849': 0.1, 'test/14852': 0.4, 'test/14854': 0.0, 'test/14858': 0.2, 'test/14859': 0.5, 'test/14860': 0.5, 'test/14861': 0.1, 'test/14862': 0.1, 'test/14863': 0.0, 'test/14865': 0.1, 'test/14867': 0.0, 'test/14872': 0.3, 'test/14873': 0.3, 'test/14875': 0.3, 'test/14876': 0.3, 'test/14877': 0.0, 'test/14881': 0.0, 'test/14882': 0.0, 'test/14885': 0.0, 'test/14886': 0.1, 'test/14888': 0.0, 'test/14890': 0.0, 'test/14891': 0.0, 'test/14892': 0.0, 'test/14899': 0.5, 'test/14900': 0.2, 'test/14903': 0.3, 'test/14904': 0.0, 'test/14907': 0.1, 'test/14909': 0.1, 'test/14911': 0.4, 'test/14912': 0.0, 'test/14913': 0.0, 'test/14918': 0.0, 'test/14919': 0.0, 'test/14921': 0.1, 'test/14922': 0.0, 'test/14923': 0.0, 'test/14926': 0.5, 'test/14928': 0.0, 'test/14930': 0.3, 'test/14931

In [90]:
# Acurácias médias para comparar
bow_count_avg_accuracy = np.average(list(bow_count_top_accuracies.values()))
print(f'Average accuracy for BoW:    {bow_count_avg_accuracy}')
tf_idf_avg_accuracy = np.average(list(tf_idf_top_accuracies.values()))
print(f'Average accuracy for TF-IDF: {tf_idf_avg_accuracy}')

Average accuracy for BoW:    0.19456118665018543
Average accuracy for TF-IDF: 0.19456118665018543
